In [1]:
import torch
import os
import sys

In [106]:
import numpy as np

In [44]:
import wandb

In [81]:
epochs = 5

In [45]:
# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="dbr",
    
#     # track hyperparameters and run metadata
#     config={
#         "learning_rate": 0.02,
#         "architecture": "GAT",
#         "dataset": "CIFAR-100",
#         "epochs": 10,
#     }
# )

In [2]:
device = "cuda:0"

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
os.getcwd()

'/home/thomas/Work/Research/uil/dbr'

In [97]:
from src.graph_construction import graph
nycgraph, rebuild_info = graph(
    "new-york",
    neighbor_radius=10,
    building_buffer=30,
    test_percent=15
)

In [98]:
rebuild_info.keys()

dict_keys(['rebuild_idx', 'recorded', 'node_data', 'footprints', 'complete_footprints', 'training_mask', 'simple_ids'])

In [113]:
training_ids = np.array(rebuild_info["simple_ids"])[rebuild_info["training_mask"].cpu()]

In [130]:
validation_batches = []

for i in range(5):
    validation_batches.append(
        np.random.choice(training_ids, 100, replace=False)
    )

In [132]:
rebuild_info["training_mask"]

tensor([ True,  True,  True,  ...,  True, False,  True], device='cuda:0')

In [140]:
# now I want to build training and validation masks
cv_masks = []
for cvbatch in validation_batches:
    training_mask = []
    validation_mask = []
    
    for building_id in rebuild_info["simple_ids"]:
        if (building_id in training_ids and building_id in cvbatch):
            training_mask.append(False)
            validation_mask.append(True)
        elif (building_id in training_ids and building_id not in cvbatch):
            training_mask.append(True)
            validation_mask.append(False)
        else:
            training_mask.append(False)
            validation_mask.append(False)
            
    cv_masks.append(
        (np.array(training_mask), np.array(validation_mask))
    )

In [141]:
sample_train, sample_val = cv_masks[0]

In [143]:
sample_val.sum()

100

In [115]:
training_ids.shape

(18106,)

In [102]:
rebuild_info["training_mask"]

torch.Size([21300])

In [120]:
rebuild_info["training_mask"].sum()

tensor(18106, device='cuda:0')

In [83]:
data = nycgraph

In [49]:
nycgraph

HeteroData(
  pano={ x=[52298, 6] },
  footprint={
    x=[21300, 6],
    y=[21300],
    train_mask=[21300],
    test_mask=[21300]
  },
  (footprint, contains, pano)={ edge_index=[2, 81554] },
  (pano, links, pano)={ edge_index=[2, 117758] },
  (pano, rev_contains, footprint)={ edge_index=[2, 81554] }
)

In [50]:
nycgraph['pano'].x.shape

torch.Size([52298, 6])

In [51]:
nycgraph['footprint'].train_mask.sum()

tensor(18106, device='cuda:0')

In [59]:
from src.model import CustomGAT

model = CustomGAT(
    hidden_channels=64, 
    out_channels=64, 
    layers=5
).to(device)

In [60]:
model

CustomGAT(
  (convs): ModuleList(
    (0): GATConv((-1, -1), 64, heads=1)
    (1): GATConv((-1, -1), 64, heads=1)
    (2): GATConv((-1, -1), 64, heads=1)
    (3): GATConv((-1, -1), 64, heads=1)
    (4): GATConv((-1, -1), 64, heads=1)
  )
  (lins): ModuleList(
    (0): Linear(-1, 64, bias=True)
    (1): Linear(-1, 64, bias=True)
    (2): Linear(-1, 64, bias=True)
    (3): Linear(-1, 64, bias=True)
    (4): Linear(-1, 64, bias=True)
  )
  (conv_translate): HeteroConv(num_relations=1)
  (convf): GATConv((-1, -1), 64, heads=1)
  (linf): Linear(-1, 64, bias=True)
  (mlp): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU()
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [88]:
epochs = 100

training_losses = []
validation_losses = []

optimizer = torch.optim.Adam(model.parameters())
loss_func = torch.nn.MSELoss()

for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)

#     print(f"Count of NaN predictions: {(~captured_terms).sum()}")
    mask = data['footprint'].train_mask

    training_predictions = out[mask].squeeze()
    training_values = data['footprint'].y[mask]

    loss = loss_func(
        training_predictions,
        training_values
    )
    loss_history.append(loss)

    if epoch % 50 == 0:
        print(f"Loss at step: {epoch} is {loss}")
    loss.backward()
    optimizer.step()

Loss at step: 0 is 27.2962646484375
Loss at step: 50 is 8.906781196594238
Loss at step: 100 is 1.3443896770477295
Loss at step: 150 is 1.2587026357650757
Loss at step: 200 is 1.2287826538085938
Loss at step: 250 is 1.214066505432129
Loss at step: 300 is 1.1923410892486572
Loss at step: 350 is 1.18222975730896
Loss at step: 400 is 1.1707290410995483
Loss at step: 450 is 1.1478931903839111
Loss at step: 500 is 1.139016032218933
Loss at step: 550 is 1.1338568925857544
Loss at step: 600 is 1.1258440017700195
Loss at step: 650 is 1.1159573793411255
Loss at step: 700 is 1.1099470853805542
Loss at step: 750 is 1.1276657581329346
Loss at step: 800 is 1.093024492263794
Loss at step: 850 is 1.0922297239303589
Loss at step: 900 is 1.095910906791687
Loss at step: 950 is 1.103584885597229


In [78]:
output = model(
    nycgraph.x_dict, 
    nycgraph.edge_index_dict
)

In [80]:
# param_size = 0
# for param in model.parameters():
#     param_size += param.nelement() * param.element_size()
# buffer_size = 0
# for buffer in model.buffers():
#     buffer_size += buffer.nelement() * buffer.element_size()

# size_all_mb = (param_size + buffer_size) / 1024**2
# print('model size: {:.3f}MB'.format(size_all_mb))

In [62]:
nycgraph.x_dict['footprint'].shape

torch.Size([21300, 6])

In [74]:
output[rebuild_info['rebuild_idx']].flatten().shape

torch.Size([45941])

In [73]:
rebuild_info['recorded'].shape

torch.Size([45941])

In [ ]:
output[]